# Playing with Contrast Enhancement, Histogram Equalization
stough 202-

We've now seen how to improve image contrast through the application of transfer functions like [windowing, power/gamma, log, and others](./enhance_histeq.ipynb). We've also seen how [Histogram Equalization](./enhance_histeq.ipynb) creates an ideal transfer function in some sense--that of leading to a uniform histogram. Use these other demos in order to answer the following questions

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatLogSlider
from scipy.interpolate import interp1d

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)

from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       vis_pair,
                       lab_uniform)

## Find an Image

Find **at least** two reasonably inoffensive images (maybe a little offensive) to test on. One key here is that because we're doing contrast enhancement, you might err on the side of lower-contrast images. You don't have to do every test on both, but it wouldn't be difficult.

In [2]:
I1 = plt.imread('../dip_pics/roadforest.jpg')
I2 = plt.imread('../dip_pics/dark_nature.jpg')

## 1. Parametric Transfer Functions 

- Use `vis_hists` to **show the histogram of an image**. 
- With that visual assistance consider what parametric transfer function (power/gamma, log, exponent) might be helpful in this image's case.
- **Justify your choice** with written description
- **Display the result** of applying the transfer function of choice to the image. 
- You can use `vis_pair` to show both the original and the presumeably contrast-enhanced images together.
- **Explain how**, with written description, the figures you generate either do or do not reflect your justification from before. You can even **show a zoomed-in section** that illustrates your contentions.

In [3]:
vis_hists(I2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

With respect to the image and its histogram we need to use non-linear methods to enhance this image since its color components are spread relatively wide from 0 to around 175 the linear transformation won't have ideal affects.

In [4]:
I2_log_corrected = ((255/8)*np.log2(1.0 + I2)).astype('uint8')
vis_hists(I2_log_corrected)
vis_pair(I2, I2_log_corrected, second_title='Log Corrected', show_ticks=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2. Alternative Colorspaces

**Show** the same parametric contrast enhancement as in Part 1, but performed **only on Luminance/Y/Value** dimension after conversion into  [Lab](../Color/color_Lab.ipynb)/[YCbCr](../Color/color_YCbCr.ipynb)/[HSV](../Color/color_HSV.ipynb). **Try all three alternative colorspaces and explain your results**. Remember, you'll need to: 

- Convert to the alternative colorspace using the appropriate [skimage.color](https://scikit-image.org/docs/dev/api/skimage.color.html) function.
- Change only the one channel you're working with. Be sure you're not producing out-of-bounds values for the particular colorspace you choose.
- Convert back using the appropriate inverse from `skimage.color`.

You can use `vis_hists` and/or extend `vis_pair` to show all three together. **Compare and contrast your results**.

In [5]:
#Convert to alternative colorspace
I2_hsv = color.rgb2hsv(I2)
I2_lab = color.rgb2lab(I2)
I2_ybr = color.rgb2ycbcr(I2)

## Lab 

In [6]:
vis_hists(I2_lab[...,0])
L,A,B = np.split(I2_lab, indices_or_sections=3,axis=-1)
temp = L.copy()
arr_info(temp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((2678, 4480, 1), dtype('float64'), 0.039573654069084085, 66.6368544813074)

In [7]:
arr_info((100/np.log2(68))*np.log2(1.0 + temp))

((2678, 4480, 1), dtype('float64'), 0.9197916866822682, 99.87309690073452)

In [8]:
L_e = (100/np.log2(68))*np.log2(1.0 + temp)
I2_lab_e = np.dstack((L_e,A,B))
I2_lab2rgb = color.lab2rgb(I2_lab_e)

In [9]:
vis_pair(I2,I2_lab2rgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
def applyGamma(I, gamma = 1):
    c = 100.0/(np.power(100.0, gamma))
    J = c*np.power(I.astype('float'), gamma)
    return J

In [39]:
J = L.copy()
arr_info(J)

((2678, 4480, 1), dtype('float64'), 0.039573654069084085, 66.6368544813074)

In [40]:
J = applyGamma(J,gamma = 0.3)

In [41]:
arr_info(J)

((2678, 4480, 1), dtype('float64'), 9.532829948570528, 88.53486849539097)

In [42]:
J2_e = np.dstack((J,A,B))
J2_f = color.lab2rgb(J2_e)

In [43]:
vis_pair(I2,J2_f)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## HSV

In [15]:
vis_hists(I2_hsv[...,2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
H,S,V = np.split(I2_hsv, indices_or_sections=3,axis=-1)
temp2 = V.copy()

In [17]:
arr_info(temp2)

((2678, 4480, 1), dtype('float64'), 0.00392156862745098, 0.7607843137254902)

In [18]:
arr_info((100/np.log2(76))*np.log2(1.0 + temp2)/13.07)

((2678, 4480, 1), dtype('float64'), 0.006914688477943239, 0.9995273987572477)

In [19]:
V_e = (100/np.log2(76))*np.log2(1.0 + temp2)/13.07

In [20]:
I2_hsv_e = np.dstack((H,S,V_e))
I2_hsv2rgb = color.hsv2rgb(I2_hsv_e)

In [21]:
vis_hists(I2_hsv2rgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
vis_pair(I2,I2_hsv2rgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Ycbcr

In [23]:
vis_hists(I2_ybr[...,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
Y,Cb,Cr = np.split(I2_ybr, indices_or_sections=3,axis=-1)
temp3 = Y.copy()

In [48]:
arr_info(temp3)

((2678, 4480, 1), dtype('float64'), 16.195811764705883, 154.15894117647056)

In [49]:
arr_info((235/np.log2(156))*np.log2(1.0 + temp3))

((2678, 4480, 1), dtype('float64'), 132.37931423936996, 234.74842701244776)

In [50]:
Y_e = (235/np.log2(156))*np.log2(1.0 + temp3)

In [51]:
arr_info(Y_e)

((2678, 4480, 1), dtype('float64'), 132.37931423936996, 234.74842701244776)

In [71]:
I2_ybr_e = (np.dstack((Y_e,Cb,Cr)))

In [72]:
arr_info(I2_ybr_e[...,2])

((2678, 4480), dtype('float64'), 97.1258274509804, 133.12773333333334)

In [90]:
def simple_linear(I, A = 1.0, B = 0.0):
    '''
    simpleLinear(I, A = 1.0, B = 0.0): return a linearly transformed image with the 
    provided constants A and B. 
    '''
    Tr = lambda x: A*x + B
    J = Tr(I.astype('float'))
    return np.clip(J, 0, 255).astype('uint8')

In [91]:
I2_ybr2rgb = color.ycbcr2rgb(I2_ybr_e)
J2 = I2_ybr2rgb.copy()

In [93]:
J_normed = simple_linear(J2, A = 255.0/(J2.max()-J2.min()), B = -255*J2.min()/(J2.max()-J2.min()))

In [82]:
arr_info(I2_ybr2rgb[...,])

((2678, 4480), dtype('float64'), 0.5305182761400187, 0.9517581506673418)

In [78]:
vis_hists(I2_ybr2rgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
vis_hists(J_normed)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
vis_pair(I2,J_normed)

../dip_utils/vis_utils.py:238: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, ax = plt.subplots(1,2, figsize=figsize, sharex = shared, sharey = shared)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [105]:
f, ax = plt.subplots(1,4,figsize=(12,6),sharex=True,sharey=True)
ax[0].imshow(I2)
ax[0].set_title('Original')
ax[1].imshow(I2_lab2rgb)
ax[1].set_title('Lab Space')
ax[2].imshow(I2_hsv2rgb)
ax[2].set_title('HSV Space')
ax[3].imshow(J_normed)
ax[3].set_title('YCbCr Space')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'YCbCr Space')

## 3. Histogram Equalization

Again you can use `vis_hists`, `vis_pair`, `vis_[rgb,lab,hsv,ybr]_cube` to visualize results.

- **Histogram-equalize an image and show** the resulting change in the image (as in, "Before" and "After" with `vis_pair`).
- **Explain how**, with written description, the figures you generate do or do not reflect good contrast enhancement. Are there artifacts in the corrected image that make it not an "improvement" in your eyes? 
- Try with two alternative colorspaces: **Perform the histogram equalization on only the Luminance/Y/Value** dimension after conversion into  [Lab](../Color/color_Lab.ipynb)/[YCbCr](../Color/color_YCbCr.ipynb)/[HSV](../Color/color_HSV.ipynb) space, and **show the results**. 
- **Explain your results**.

In [103]:
# I2 = plt.imread('../dip_pics/dark_nature.jpg')